In [1]:
!pip install catboost
!pip install xgboost
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from imblearn.combine import SMOTEENN, SMOTETomek
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_auc_score, roc_curve, precision_score,recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV



/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
#Loading Dataset
data= pd.read_csv('/content/Telco_Customer_Churn.csv')
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
# Checking Data Types

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
#Converting Object to numeric

data["TotalCharges"] = pd.to_numeric(data["TotalCharges"], errors="coerce")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [6]:
#Checking null values

data.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [7]:
#Checking Duplicate values

data.duplicated().sum()

0

In [8]:
#Droping Customer ID

data.drop("customerID", axis=1, inplace=True)

In [9]:
data

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [10]:
#spliting to X_train and Y_train

X = data.drop("Churn", axis=1)
Y = data["Churn"]

In [11]:
#Checking Shape

X.shape

(7043, 19)

In [12]:
Y.shape

(7043,)

In [13]:
Y = np.where(Y == "Yes", 1, 0)
Y

array([0, 0, 1, ..., 0, 1, 0])

In [14]:

# Data Transformation in Pipline and ColumnTransformer

# Spliting Numeric Features and Categorical Features


numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include="object").columns.tolist()

num_pipline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
cat_pipline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder())
])

preprocessor = ColumnTransformer(transformers=[
    ("num", num_pipline, numeric_features),
    ("cat", cat_pipline, categorical_features)
])

In [15]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['SeniorCitizen', 'tenure', 'MonthlyCharges',
                                  'TotalCharges']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OrdinalEncoder())]),
                                 ['gender', 'Partner', 'Dependents',
                                  'PhoneService', 'MultipleLines',
                                  'InternetService', 'OnlineSecurity',
                                  'OnlineBackup', 'DeviceProtection',
                                  'TechSupport', 'StreamingTV',
                                  'StreamingMovies', 'Contract',
                                  'PaperlessBilling', 'PaymentMethod'])])

In [16]:
#Fiting preprocessor to X_train

X_pre_transformed = preprocessor.fit_transform(X)

In [17]:
#resamping data to get Balanced Data

smt = SMOTEENN()
X_resampled, y_resampled = smt.fit_resample(X_pre_transformed, Y )
X_resampled.shape, y_resampled.shape

((6384, 19), (6384,))

In [18]:
#Spliting Train and Test

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5107, 19), (1277, 19), (5107,), (1277,))

In [19]:
### Evaluation Metrics ###
def evaluate_clf(true, predicted):
    acc = accuracy_score(true, predicted)
    f1 = f1_score(true, predicted)
    precision = precision_score(true, predicted)
    recall = recall_score(true, predicted)
    roc_auc = roc_auc_score(true, predicted)
    return acc, f1 , precision, recall, roc_auc

In [20]:
def evaluate_models(X, y, models, params):
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    results = []

    for model_name, model in models.items():
        # Perform Grid Search for hyperparameter tuning
        gs = GridSearchCV(model, params[model_name], cv=3)
        gs.fit(X_train, y_train)

        # Set the best parameters and fit the model
        model.set_params(**gs.best_params_)
        model.fit(X_train, y_train)

        # Make predictions
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Evaluate performance on training and test sets
        train_metrics = evaluate_clf(y_train, y_train_pred)
        test_metrics = evaluate_clf(y_test, y_test_pred)

        # Store results
        result_entry = {
            'Model Name': model_name,
            'Train Accuracy': train_metrics[0],
            'Test Accuracy': test_metrics[0],
            'Train F1 Score': train_metrics[1],
            'Test F1 Score': test_metrics[1],
            'Train Precision': train_metrics[2],
            'Test Precision': test_metrics[2],
            'Train Recall': train_metrics[3],
            'Test Recall': test_metrics[3],
            'Train Roc Auc Score': train_metrics[4],
            'Test Roc Auc Score': test_metrics[4]
        }

        results.append(result_entry)

        # Print the results for the current model
        print(f"Results for {model_name}:")
        for metric_name, value in result_entry.items():
            print(f"{metric_name}: {value:.4}")
        print("=" * 40)

    # Create a DataFrame from results
    report = pd.DataFrame(results).sort_values(by='Test Accuracy', ascending=False)

    return report

In [21]:

models = {
    'LogisticRegression': LogisticRegression(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'SVC': SVC(),
    'RandomForestClassifier': RandomForestClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'XGBClassifier': XGBClassifier(),
    'LGBMClassifier': LGBMClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'GaussianNB': GaussianNB()
}

### Define Params ###

param_grids = {
    'LogisticRegression': {
        "class_weight":["balanced"],
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'solver': ['liblinear', 'saga']
    },
    'KNeighborsClassifier': {
        'n_neighbors': [3, 5, 7, 9]
    },
    'SVC': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto']
    },
    'RandomForestClassifier': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    },
    'GradientBoostingClassifier': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5],
        'min_samples_split': [2]
    },
    'AdaBoostClassifier': {
        'n_estimators': [50, 100],
        'learning_rate': [0.01, 0.1]
    },
    'DecisionTreeClassifier': {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2]
    },
    'GaussianNB': {},
    'XGBClassifier': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 6]
    },
    'LGBMClassifier': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'num_leaves': [31, 63]
    }}

In [22]:
report = evaluate_models(X_resampled, y_resampled, models, param_grids)

Results for LogisticRegression:
Model Name: Logi
Train Accuracy: 0.9123
Test Accuracy: 0.9084
Train F1 Score: 0.9216
Test F1 Score: 0.9167
Train Precision: 0.9229
Test Precision: 0.9253
Train Recall: 0.9203
Test Recall: 0.9083
Train Roc Auc Score: 0.9112
Test Roc Auc Score: 0.9084
Results for KNeighborsClassifier:
Model Name: KNei
Train Accuracy: 0.9881
Test Accuracy: 0.9648
Train F1 Score: 0.9894
Test F1 Score: 0.9691
Train Precision: 0.9821
Test Precision: 0.9439
Train Recall: 0.9969
Test Recall: 0.9958
Train Roc Auc Score: 0.9869
Test Roc Auc Score: 0.9609
Results for SVC:
Model Name: SVC
Train Accuracy: 0.9832
Test Accuracy: 0.9483
Train F1 Score: 0.985
Test F1 Score: 0.9541
Train Precision: 0.9853
Test Precision: 0.941
Train Recall: 0.9846
Test Recall: 0.9676
Train Roc Auc Score: 0.983
Test Roc Auc Score: 0.9459
Results for RandomForestClassifier:
Model Name: Rand
Train Accuracy: 1.0
Test Accuracy: 0.9601
Train F1 Score: 1.0
Test F1 Score: 0.9643
Train Precision: 1.0
Test Precisio

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

Results for AdaBoostClassifier:
Model Name: AdaB
Train Accuracy: 0.9178
Test Accuracy: 0.9186
Train F1 Score: 0.9276
Test F1 Score: 0.9279
Train Precision: 0.915
Test Precision: 0.9127
Train Recall: 0.9406
Test Recall: 0.9436
Train Roc Auc Score: 0.9146
Test Roc Auc Score: 0.9155
Results for XGBClassifier:
Model Name: XGBC
Train Accuracy: 0.9992
Test Accuracy: 0.9608
Train F1 Score: 0.9993
Test F1 Score: 0.9647
Train Precision: 0.9986
Test Precision: 0.9661
Train Recall: 1.0
Test Recall: 0.9633
Train Roc Auc Score: 0.9991
Test Roc Auc Score: 0.9605
[LightGBM] [Info] Number of positive: 1906, number of negative: 1498
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001139 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1865
[LightGBM] [Info] Number of data points in the train set: 3404, number of used features: 19
[LightGBM] [Info] [binar

In [23]:
import pickle
pickle.dump(models, open('model2.pkl', 'wb'))

In [24]:
report

,Model Name,Train Accuracy,Test Accuracy,Train F1 Score,Test F1 Score,Train Precision,Test Precision,Train Recall,Test Recall,Train Roc Auc Score,Test Roc Auc Score
1,KNeighborsClassifier,0.988056,0.964761,0.989415,0.969115,0.982087,0.943850,0.996853,0.995769,0.986856,0.960913
7,LGBMClassifier,1.000000,0.963978,1.000000,0.967651,1.000000,0.964937,1.000000,0.970381,1.000000,0.963183
6,XGBClassifier,0.999217,0.960846,0.999301,0.964689,0.998603,0.966054,1.000000,0.963329,0.999110,0.960538
3,RandomForestClassifier,1.000000,0.960063,1.000000,0.964311,1.000000,0.956944,1.000000,0.971791,1.000000,0.958607
4,GradientBoostingClassifier,0.999021,0.956930,0.999126,0.961295,0.998951,0.959270,0.999301,0.963329,0.998983,0.956136
2,SVC,0.983160,0.948316,0.984960,0.954103,0.985304,0.941015,0.984615,0.967560,0.982962,0.945928
8,DecisionTreeClassifier,0.987468,0.929522,0.988835,0.936709,0.986769,0.934081,0.990909,0.939351,0.986999,0.928302
5,AdaBoostClassifier,0.917760,0.918559,0.927586,0.927878,0.914966,0.912688,0.940559,0.943583,0.914650,0.915453
0,LogisticRegression,0.912277,0.908379,0.921569,0.916726,0.922861,0.925287,0.920280,0.908322,0.911186,0.908386
9,GaussianNB,0.900529,0.895850,0.909318,0.904110,0.928884,0.924779,0.890559,0.884344,0.901889,0.897278


#- Hyperparameter Tuning: Each model undergoes grid search for hyperparameter optimization. The best combination of hyperparameters is used to train the final model.
#- Model Evaluation: For each model, performance is assessed using multiple metrics on both the training and test datasets.
#- Result Compilation: The performance results of all models are compiled into a DataFrame and sorted by test accuracy for easy comparison.



In [25]:
X_1 = X[['gender', 'InternetService', 'Contract', 'tenure', 'MonthlyCharges', 'TotalCharges']]
y_1 =Y

In [26]:
X_1.head(5)

,gender,InternetService,Contract,tenure,MonthlyCharges,TotalCharges
0,Female,DSL,Month-to-month,1,29.85,29.85
1,Male,DSL,One year,34,56.95,1889.50
2,Male,DSL,Month-to-month,2,53.85,108.15
3,Male,DSL,One year,45,42.30,1840.75
4,Female,Fiber optic,Month-to-month,2,70.70,151.65


In [27]:
### Data Transformation in Pipline and ColumnTransformer ###

### Spliting Numeric Features and Categorical Features ###

numeric_1= ['tenure', 'MonthlyCharges', 'TotalCharges']
categorical_1= ['gender', 'InternetService', 'Contract']

### Numeric Pipline ###

num_pipline_1 = Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', StandardScaler())
])

### Categorical Pipline ###

cat_pipline_1 = Pipeline([
    ('encoder', OrdinalEncoder())
])

### ColumnTransformer Object ###
preprosser_1 = ColumnTransformer([
    ('numeric', num_pipline_1, numeric_1),
    ('categorical', cat_pipline_1, categorical_1)
])
preprosser_1

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['tenure', 'MonthlyCharges', 'TotalCharges']),
                                ('categorical',
                                 Pipeline(steps=[('encoder',
                                                  OrdinalEncoder())]),
                                 ['gender', 'InternetService', 'Contract'])])

In [28]:
## Fiting Preprosser Object in X Features ###

X_pre_transformed_1 = preprosser_1.fit_transform(X_1)

### Checking X_pre_transformed ###

X_pre_transformed_1.shape

(7043, 6)

In [29]:
import pickle
pickle.dump(preprosser_1, open('preprocessor2.pkl', 'wb'))

In [30]:
### Printing X_pre_transformed Values ###

X_pre_transformed_1

array([[-1.27744458, -1.16032292, -0.99424194,  0.        ,  0.        ,
         0.        ],
       [ 0.06632742, -0.25962894, -0.17324413,  1.        ,  0.        ,
         1.        ],
       [-1.23672422, -0.36266036, -0.95967407,  1.        ,  0.        ,
         0.        ],
       ...,
       [-0.87024095, -1.1686319 , -0.85446945,  0.        ,  0.        ,
         0.        ],
       [-1.15528349,  0.32033821, -0.87206242,  1.        ,  1.        ,
         0.        ],
       [ 1.36937906,  1.35896134,  2.01428802,  1.        ,  1.        ,
         2.        ]])

In [31]:
### y target Value Encoding ###

le = LabelEncoder()
y_encoded_1 = le.fit_transform(y_1)
y_encoded_1

### Checking y_encoded Shape ###
#y_encoded.shape

array([0, 0, 1, ..., 0, 1, 0])

In [32]:
smt_1 = SMOTEENN()
X_resampled_1, y_resampled_1 = smt_1.fit_resample(X_pre_transformed_1, y_encoded_1)
X_resampled_1.shape, y_resampled_1.shape

((6029, 6), (6029,))

In [33]:
report_2 = evaluate_models(X_resampled_1, y_resampled_1, models, param_grids)

Results for LogisticRegression:
Model Name: Logi
Train Accuracy: 0.9032
Test Accuracy: 0.8872
Train F1 Score: 0.9077
Test F1 Score: 0.8891
Train Precision: 0.8847
Test Precision: 0.8596
Train Recall: 0.9318
Test Recall: 0.9206
Train Roc Auc Score: 0.9025
Test Roc Auc Score: 0.8878
Results for KNeighborsClassifier:
Model Name: KNei
Train Accuracy: 0.9944
Test Accuracy: 0.971
Train F1 Score: 0.9945
Test F1 Score: 0.9707
Train Precision: 0.9919
Test Precision: 0.9619
Train Recall: 0.9972
Test Recall: 0.9797
Train Roc Auc Score: 0.9943
Test Roc Auc Score: 0.9711
Results for SVC:
Model Name: SVC
Train Accuracy: 0.9212
Test Accuracy: 0.9063
Train F1 Score: 0.9221
Test F1 Score: 0.9061
Train Precision: 0.9309
Test Precision: 0.892
Train Recall: 0.9135
Test Recall: 0.9206
Train Roc Auc Score: 0.9214
Test Roc Auc Score: 0.9066
Results for RandomForestClassifier:
Model Name: Rand
Train Accuracy: 1.0
Test Accuracy: 0.9701
Train F1 Score: 1.0
Test F1 Score: 0.9698
Train Precision: 1.0
Test Precisi

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

Results for AdaBoostClassifier:
Model Name: AdaB
Train Accuracy: 0.9075
Test Accuracy: 0.8872
Train F1 Score: 0.9104
Test F1 Score: 0.8882
Train Precision: 0.9007
Test Precision: 0.8654
Train Recall: 0.9204
Test Recall: 0.9122
Train Roc Auc Score: 0.9072
Test Roc Auc Score: 0.8877
Results for XGBClassifier:
Model Name: XGBC
Train Accuracy: 0.9913
Test Accuracy: 0.9602
Train F1 Score: 0.9915
Test F1 Score: 0.9601
Train Precision: 0.9871
Test Precision: 0.9444
Train Recall: 0.9959
Test Recall: 0.9764
Train Roc Auc Score: 0.9912
Test Roc Auc Score: 0.9605
[LightGBM] [Info] Number of positive: 1642, number of negative: 1573
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000152 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 775
[LightGBM] [Info] Number of data points in the train set: 3215, number of used features: 6
[LightGBM] [Info] [bin

In [34]:
report_2

,Model Name,Train Accuracy,Test Accuracy,Train F1 Score,Test F1 Score,Train Precision,Test Precision,Train Recall,Test Recall,Train Roc Auc Score,Test Roc Auc Score
1,KNeighborsClassifier,0.994402,0.970978,0.994533,0.970711,0.991922,0.961857,0.997158,0.979730,0.994342,0.971135
3,RandomForestClassifier,1.000000,0.970149,1.000000,0.969849,1.000000,0.961794,1.000000,0.978041,1.000000,0.970291
7,LGBMClassifier,0.999793,0.967662,0.999797,0.967473,0.999594,0.955519,1.000000,0.979730,0.999788,0.967878
6,XGBClassifier,0.991292,0.960199,0.991512,0.960133,0.987123,0.944444,0.995940,0.976351,0.991190,0.960488
8,DecisionTreeClassifier,1.000000,0.953566,1.000000,0.953020,1.000000,0.946667,1.000000,0.959459,1.000000,0.953671
4,GradientBoostingClassifier,0.997512,0.951907,0.997569,0.952066,0.995552,0.932039,0.999594,0.972973,0.997466,0.952285
2,SVC,0.921211,0.906302,0.922131,0.906068,0.930906,0.891980,0.913520,0.920608,0.921379,0.906558
0,LogisticRegression,0.903172,0.887231,0.907653,0.889070,0.884734,0.859621,0.931790,0.920608,0.902548,0.887828
5,AdaBoostClassifier,0.907526,0.887231,0.910442,0.888158,0.900675,0.865385,0.920422,0.912162,0.907245,0.887677
9,GaussianNB,0.888451,0.873964,0.891576,0.874380,0.885154,0.855987,0.898092,0.893581,0.888241,0.874315
